In [1]:
import altair as alt


def make_bars_chart(df, title, scale_title=None, avg_col='avg', sqrt_scale=True,
                   width=50, height=300, multi_bar=True, legend=True, labels=True):
    source = df.copy()

    if source[avg_col].max() > 1:
        avg_scale = 'secs'
        avg_coef = 1
    elif source[avg_col].max() > 0.001:
        avg_scale = 'ms'
        avg_coef = 1e3
    else:
        avg_scale = 'µs'
        avg_coef = 1e6

    source[avg_scale] = (df[avg_col] * avg_coef).round(2)
    
    if not scale_title:
        scale_titles = {
            'secs': 'seconds',
            'ms': 'milliseconds (1e−3 secs)',
            'µs': 'microseconds (1e−6 secs)',
        }
        scale_title = scale_titles[avg_scale]

    if sqrt_scale:
        y_scale = alt.Scale(type='sqrt')
    else:
        y_scale = alt.Scale()
    
    if multi_bar:
        x_val = 'bench:N'
        facet_kwds = {'column':'name:N'}
    else:
        x_val = 'name:N'
        facet_kwds = {}

    if legend:
        legend = alt.Legend()
    else:
        legend = None

    chart = alt.Chart(
        width=width,
        height=height,
    ).mark_bar(
        stroke='transparent',
        size=20,
    ).encode(
        alt.X(x_val, scale=alt.Scale(), axis=alt.Axis(title='', labels=labels)),
        alt.Y(f'{avg_scale}:Q', scale=y_scale, axis=alt.Axis(title=scale_title, grid=False)),
        color=alt.Color(x_val, scale=alt.Scale(range=["#FF7B06", "#094AFB", "#D60000"]), legend=legend),
    )

    text = chart.mark_text(
        color='black',
        dx = 0,
        dy = -2,
    ).encode(
        text=f'{avg_scale}:Q'
    )

    return alt.layer(chart, text, data=source).facet(
        **facet_kwds
    ).configure_axis(
        domainWidth=0.8,
    ).configure_view(
        stroke='transparent'
    ).properties(
        title=title
    )

In [2]:
import pandas as pd
single_df = pd.read_json('bench_results.json', orient='records')
single_df.head()

,bench,max,mean,min,std,tool
0,1-8digits,0.579587,0.552565,0.534739,0.008315,cracken
1,1-8digits,0.746159,0.713473,0.699529,0.007687,maskprocessor
2,9digits,5.053456,4.850217,4.703487,0.088734,cracken
3,9digits,178.344897,178.344897,178.344897,NaN,crunch
4,9digits,6.392499,6.257290,6.164827,0.074914,maskprocessor


In [3]:
df = single_df.groupby(['bench', 'tool']).mean().reset_index()
df['tool'].replace('crunch', ' crunch', inplace=True)
df

,bench,tool,max,mean,min,std
0,1-8digits,cracken,0.579587,0.552565,0.534739,0.008315
1,1-8digits,maskprocessor,0.746159,0.713473,0.699529,0.007687
2,9digits,cracken,5.053456,4.850217,4.703487,0.088734
3,9digits,crunch,178.344897,178.344897,178.344897,NaN
4,9digits,maskprocessor,6.392499,6.257290,6.164827,0.074914
5,upper-5lower-digit,cracken,15.319965,14.976077,14.625821,0.231255
6,upper-5lower-digit,crunch,465.233578,465.233578,465.233578,NaN
7,upper-5lower-digit,maskprocessor,20.000896,19.763502,19.486918,0.231909


In [4]:
df['secs'] = df['mean']
df['name'] = df['tool']
df['ms'] = df['mean'] * 1000.0
df['µs'] = df['mean'] * 1_000_000.0
df

,bench,tool,max,mean,min,std,secs,name,ms,µs
0,1-8digits,cracken,0.579587,0.552565,0.534739,0.008315,0.552565,cracken,552.565017,5.525650e+05
1,1-8digits,maskprocessor,0.746159,0.713473,0.699529,0.007687,0.713473,maskprocessor,713.472804,7.134728e+05
2,9digits,cracken,5.053456,4.850217,4.703487,0.088734,4.850217,cracken,4850.216589,4.850217e+06
3,9digits,crunch,178.344897,178.344897,178.344897,NaN,178.344897,crunch,178344.897032,1.783449e+08
4,9digits,maskprocessor,6.392499,6.257290,6.164827,0.074914,6.257290,maskprocessor,6257.290304,6.257290e+06
5,upper-5lower-digit,cracken,15.319965,14.976077,14.625821,0.231255,14.976077,cracken,14976.077292,1.497608e+07
6,upper-5lower-digit,crunch,465.233578,465.233578,465.233578,NaN,465.233578,crunch,465233.577728,4.652336e+08
7,upper-5lower-digit,maskprocessor,20.000896,19.763502,19.486918,0.231909,19.763502,maskprocessor,19763.502462,1.976350e+07


In [5]:
make_bars_chart(
    df,
    title='Wordlist Generation Time',
    sqrt_scale=True,
    labels=False,
    avg_col='mean',
    width=150,
    height=300,
)

alt.FacetChart(...)

In [6]:
min_took_df = df[df['tool'] == 'cracken'][['bench', 'mean']].groupby('bench').min().reset_index()
min_took = {x.bench: x.mean for x in min_took_df.itertuples()}
min_took

{'1-8digits': 0.55256501687776,
 '9digits': 4.850216588973999,
 'upper-5lower-digit': 14.976077291700575}

In [7]:
df['speedup'] = df.apply(lambda x: x['mean'] / min_took[x.bench], axis=1)

In [8]:
make_bars_chart(
    df,
    title='Wordlist Generation Time Speedup',
    sqrt_scale=True,
    labels=False,
    avg_col='speedup',
    width=150,
    height=300,
)

alt.FacetChart(...)

In [9]:
df.groupby('tool').mean()[['speedup']]

,speedup
tool,
crunch,33.917809
cracken,1.000000
maskprocessor,1.300326
